In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import pickle
import jax
import jax.numpy as jnp
import bayes3d as b
import genjax
import matplotlib.pyplot as plt

In [3]:
console = genjax.console(enforce_checkify=True)

In [4]:
b.setup_visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [5]:
with open('../data.pkl', 'rb') as f:
    data = pickle.load(f)

camera_image_1 = data["init"][0]
categories_on_table = data["init"][1]
target_category = data["init"][2]
table_info = data["init"][3] # table pose and dimensions
n_objects = 5

X_WT = b.t3d.pybullet_pose_to_transform(table_info[0])
X_WC = b.t3d.pybullet_pose_to_transform(camera_image_1["camera_pose"])
X_CT = b.t3d.inverse_pose(X_WC) @ X_WT

def image_to_rgbd(camera_image_1):
    K = camera_image_1['camera_matrix']
    rgb = camera_image_1['rgbPixels']
    depth = camera_image_1['depthPixels']
    camera_pose = camera_image_1['camera_pose']
    camera_pose = b.t3d.pybullet_pose_to_transform(camera_pose)
    fx, fy, cx, cy = K[0,0],K[1,1],K[0,2],K[1,2]
    h,w = depth.shape
    rgbd_original = b.RGBD(rgb, depth, camera_pose, b.Intrinsics(h,w,fx,fy,cx,cy,0.001,10000.0))
    return rgbd_original

rgbd_original = image_to_rgbd(camera_image_1)
print("Got rgbd_original")

scaling_factor = 0.2
rgbd_scaled_down = b.RGBD.scale_rgbd(rgbd_original, scaling_factor)
print("Got rgb immage scaled down.")

b.setup_renderer(rgbd_scaled_down.intrinsics)

# os.path.join(os.path.abspath(''), 'bayes3d/assets/bop/ycbv/models')# 
model_dir = os.path.join(os.path.abspath('../..'), 'bayes3d/assets/bop/ycbv/models')
mesh_path = os.path.join(model_dir,"obj_" + "{}".format(13+1).rjust(6, '0') + ".ply")
ycb_filenames = glob.glob(os.path.join(model_dir, "*.ply"))
ycb_index_order = [int(s.split("/")[-1].split("_")[-1].split(".")[0]) for s in ycb_filenames]
sorted_ycb_filenames = [s for _,s in sorted(zip(ycb_index_order, ycb_filenames))]

relevant_objects = [any(x in name for x in categories_on_table) for (i, name) in enumerate(b.utils.ycb_loader.MODEL_NAMES)]
relevant_object_names = [b.utils.ycb_loader.MODEL_NAMES[i] for i in range(len(b.utils.ycb_loader.MODEL_NAMES)) if relevant_objects[i]]
filtered_filenames = [sorted_ycb_filenames[i] for i in range(len(sorted_ycb_filenames)) if relevant_objects[i]]

table_dims = table_info[1:]
table_mesh = b.utils.make_cuboid_mesh(table_dims)
b.RENDERER.add_mesh(table_mesh, "table")
print("Added table mesh.")

pillar_mesh = b.utils.make_cuboid_mesh(jnp.array([0.02, 0.02, 0.5]))
b.RENDERER.add_mesh(pillar_mesh, "pillar")
print("Added pillar mesh.")

for model_path in filtered_filenames:
    b.RENDERER.add_mesh_from_file(model_path, scaling_factor=1.0/1000.0)
    print(f"Added mesh at path {model_path}.")

Got rgbd_original
Got rgb immage scaled down.


[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (64, 64, 1024)
Added table mesh.
Added pillar mesh.
Added mesh at path /home/georgematheos/bayes3d/assets/bop/ycbv/models/obj_000002.ply.
Added mesh at path /home/georgematheos/bayes3d/assets/bop/ycbv/models/obj_000003.ply.
Added mesh at path /home/georgematheos/bayes3d/assets/bop/ycbv/models/obj_000011.ply.
Added mesh at path /home/georgematheos/bayes3d/assets/bop/ycbv/models/obj_000013.ply.
Added mesh at path /home/georgematheos/bayes3d/assets/bop/ycbv/models/obj_000021.ply.


# Start using the model

In [6]:
from src.model2 import model, viz_trace_meshcat

In [7]:
importance_jit = jax.jit(model.importance)
key = jax.random.PRNGKey(0)

In [8]:
genjax.indexed_choice_map(
    [0, 1],
    genjax.choice_map({"a": jnp.array([3, 2])})
)

IndexedChoiceMap(
  indices=i32[2],
  inner=HierarchicalChoiceMap(trie=Trie(inner={'a': ChoiceValue(value=i32[2])}))
)

In [9]:
def flat_choicemaps_to_vector_choicemap(choicemaps):
    cm = genjax.choice_map({
        k : jnp.array([c.get_submap(k).get_value() for c in choicemaps])
        for (k, _) in choicemaps[0].get_submaps_shallow()
    })
    return cm

cm = flat_choicemaps_to_vector_choicemap([
    genjax.choice_map({"a": jnp.array([3, 2]), "b": jnp.array([1, 0])}),
    genjax.choice_map({"a": jnp.array([5, 4]), "b": jnp.array([3, 2])}),
])
console.print(cm)

(HierarchicalChoiceMap)
├── :a
│   └── (ValueChoice)
│       └──  i32[2,2]
└── :b
    └── (ValueChoice)
        └──  i32[2,2]

In [10]:
cm = genjax.choice_map({"a": jnp.array([3, 2])})
cm.get_submap("a")

ChoiceValue(value=i32[2])

In [11]:
map_choicemap = genjax.indexed_choice_map(
            jnp.array([0, 1]),
            flat_choicemaps_to_vector_choicemap([
                genjax.choice_map({
                    "root_pose": X_CT,
                    "category_index": 0,
                    "parent_obj": -1,
                    "face_parent": 2,
                    "face_child": 3,
                    "contact_params": jnp.zeros(3)
                }),
                genjax.choice_map({
                    "root_pose": jnp.eye(4),
                    "parent_obj": 0,
                    "category_index": 2,
                    "face_parent": 2,
                    "face_child": 3,
                    "contact_params": jnp.zeros(3)
                })
            ])
        )

console.print(map_choicemap)

(IndexedChoiceMap,  i32[2])
└── (HierarchicalChoiceMap)
    ├── :root_pose
    │   └── (ValueChoice)
    │       └──  f32[2,4,4]
    ├── :category_index
    │   └── (ValueChoice)
    │       └──  i32[2]
    ├── :parent_obj
    │   └── (ValueChoice)
    │       └──  i32[2]
    ├── :face_parent
    │   └── (ValueChoice)
    │       └──  i32[2]
    ├── :face_child
    │   └── (ValueChoice)
    │       └──  i32[2]
    └── :contact_params
        └── (ValueChoice)
            └──  f32[2,3]

In [31]:
trace, _ = importance_jit(key, genjax.choice_map({
        "camera_pose": jnp.eye(4),
        "objects": map_choicemap,
        # ("objects", 0): genjax.choice_map({
        #     "root_pose": X_CT,
        #     "category_index": 0,
        #     "parent_obj": -1
        # }),
        # ("objects", 1): genjax.choice_map({
        #     "parent_obj": -0,
        #     "category_index": 2,
        #     "face_parent": 2,
        #     "face_child": 3,
        #     "contact_params": jnp.zeros(3)
        # })
    }), (
        genjax.const(5), # max_n_objects
        5,               # n_objects
        # possible_object_indices
        jnp.arange(len(b.RENDERER.meshes)),
        # pose_bounds
        jnp.array([-jnp.ones(3)*5.0, jnp.ones(3)*5.0]),
        # contact_bounds
        jnp.array([jnp.array([-2, -2, -jnp.pi]), jnp.array([2.0, 2.0, jnp.pi])]),
        # all_box_dims
        b.RENDERER.model_box_dims
    )
)

Got rendered.
Traced<ShapedArray(float32[48,64,3])>with<DynamicJaxprTrace(level=2/0)>
Got noisy image.
<class 'jax._src.interpreters.batching.BatchTracer'>


In [ ]:
genjax.const(5).const

5

# Walk through the model...

In [13]:
from src.model2 import generate_objects

In [14]:
(
    max_n_objects,
    n_objects,
    possible_object_indices,
    pose_bounds,
    contact_bounds,
    all_box_dims
) = (
    genjax.const(2), # max_n_objects
    2,               # n_objects
    # possible_object_indices
    jnp.arange(len(b.RENDERER.meshes)),
    # pose_bounds
    jnp.array([-jnp.ones(3)*5.0, jnp.ones(3)*5.0]),
    # contact_bounds
    jnp.array([jnp.array([-2, -2, -jnp.pi]), jnp.array([2.0, 2.0, jnp.pi])]),
    # all_box_dims
    b.RENDERER.model_box_dims
)

In [15]:
jnp.repeat(pose_bounds[None, ...], 5)

Array([-5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5., -5.,
       -5., -5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.], dtype=float32)

In [16]:
generate_objects_args = (
    jnp.arange(max_n_objects.const) < n_objects,
    (
        jnp.arange(max_n_objects.const),
        jnp.repeat(possible_object_indices[None, ...], max_n_objects.const, 0),
        jnp.repeat(pose_bounds[None, ...], max_n_objects.const, 0),
        jnp.repeat(contact_bounds[None, ...], max_n_objects.const, 0),
    )
)

In [17]:
tr = generate_objects.simulate(
    key,
    generate_objects_args
)
masked_object_info = tr.get_retval()
console.print(masked_object_info)

(Mask,  bool[2])
└── ObjectInfo
    ├── parent_obj
    │   └──  i32[2]
    ├── parent_face
    │   └──  i32[2]
    ├── child_face
    │   └──  i32[2]
    ├── category_index
    │   └──  i32[2]
    ├── pose
    │   └──  f32[2,4,4]
    └── params
        └──  f32[2,3]

In [18]:
console.print(tr.strip())

(VectorChoiceMap)
└── (Mask,  bool[2])
    └── (HierarchicalChoiceMap)
        ├── :root_pose
        │   └── (ValueChoice)
        │       └──  f32[2,4,4]
        ├── :contact_params
        │   └── (ValueChoice)
        │       └──  f32[2,3]
        ├── :parent_obj
        │   └── (ValueChoice)
        │       └──  i32[2]
        ├── :face_parent
        │   └── (ValueChoice)
        │       └──  i32[2]
        ├── :category_index
        │   └── (ValueChoice)
        │       └──  i32[2]
        └── :face_child
            └── (ValueChoice)
                └──  i32[2]

In [19]:
console.print(map_choicemap)

(IndexedChoiceMap,  i32[2])
└── (HierarchicalChoiceMap)
    ├── :root_pose
    │   └── (ValueChoice)
    │       └──  f32[2,4,4]
    ├── :category_index
    │   └── (ValueChoice)
    │       └──  i32[2]
    ├── :parent_obj
    │   └── (ValueChoice)
    │       └──  i32[2]
    ├── :face_parent
    │   └── (ValueChoice)
    │       └──  i32[2]
    ├── :face_child
    │   └── (ValueChoice)
    │       └──  i32[2]
    └── :contact_params
        └── (ValueChoice)
            └──  f32[2,3]

In [20]:
from src.model2 import generate_object

In [21]:
mask_cm = tr.get_choices().get_submap(0).unmask()
console.print(mask_cm)

(HierarchicalChoiceMap)
├── :root_pose
│   └── (ValueChoice)
│       └──  f32[4,4]
├── :contact_params
│   └── (ValueChoice)
│       └──  f32[3]
├── :parent_obj
│   └── (ValueChoice)
│       └──  i32[]
├── :face_parent
│   └── (ValueChoice)
│       └──  i32[]
├── :category_index
│   └── (ValueChoice)
│       └──  i32[]
└── :face_child
    └── (ValueChoice)
        └──  i32[]

In [22]:
mask_args = (
    True, (0, possible_object_indices, pose_bounds, contact_bounds)
)

In [23]:
masked = genjax.masking_combinator(generate_object)

In [24]:
isinstance(mask_cm, genjax.Choice)

True

In [25]:
type(mask_cm)

genjax._src.core.datatypes.generative.HierarchicalChoiceMap

In [26]:
tr_, weight = masked.importance(key, mask_cm, mask_args)

<class 'jaxlib.xla_extension.ArrayImpl'>


In [ ]:
type(weight)

jaxlib.xla_extension.ArrayImpl

In [ ]:
from genjax._src.generative_functions.combinators.masking_combinator import MaskingTrace

In [ ]:
(check, inner_args) = mask_args
tr_, w_ = masked.inner.importance(key, mask_cm, inner_args)
w_ = check * w_
MaskingTrace(masked, tr_, check)

MaskingTrace(
  mask_combinator=MaskingCombinator(
    inner=StaticGenerativeFunction(source=<function generate_object>)
  ),
  inner=StaticTrace(
    gen_fn=StaticGenerativeFunction(source=<function generate_object>),
    args=(0, i32[7], f32[2,3], f32[2,3]),
    retval=ObjectInfo(
      parent_obj=i32[],
      parent_face=i32[],
      child_face=i32[],
      category_index=i32[],
      pose=f32[4,4],
      params=f32[3]
    ),
    address_choices=Trie(
      inner={
        'parent_obj':
        DistributionTrace(
          gen_fn=UniformDiscrete(),
          args=(-1, 0),
          value=i32[],
          score=f32[]
        ),
        'face_parent':
        DistributionTrace(
          gen_fn=UniformDiscrete(),
          args=(0, 6),
          value=i32[],
          score=f32[]
        ),
        'face_child':
        DistributionTrace(
          gen_fn=UniformDiscrete(),
          args=(0, 6),
          value=i32[],
          score=f32[]
        ),
        'category_index':
       

In [29]:
isinstance(map_choicemap, genjax.Choice)

True

In [30]:
generate_objects.importance(
    key,
    map_choicemap,
    generate_objects_args
)

<class 'jax._src.interpreters.batching.BatchTracer'>


(MapTrace(
   gen_fn=MapCombinator(
     kernel=MaskingCombinator(
       inner=StaticGenerativeFunction(source=<function generate_object>)
     ),
     in_axes=(0, (0, 0, 0, 0))
   ),
   inner=MaskingTrace(
     mask_combinator=MaskingCombinator(
       inner=StaticGenerativeFunction(source=<function generate_object>)
     ),
     inner=StaticTrace(
       gen_fn=StaticGenerativeFunction(source=<function generate_object>),
       args=(i32[2], i32[2,7], f32[2,2,3], f32[2,2,3]),
       retval=ObjectInfo(
         parent_obj=i32[2],
         parent_face=i32[2],
         child_face=i32[2],
         category_index=i32[2],
         pose=f32[2,4,4],
         params=f32[2,3]
       ),
       address_choices=Trie(
         inner={
           'root_pose':
           DistributionTrace(
             gen_fn=UniformPose(),
             args=(f32[2,3], f32[2,3]),
             value=f32[2,4,4],
             score=f32[2]
           ),
           'contact_params':
           DistributionTrace(
       

In [ ]:
from src.model2 import empty_object_info
from src.masking_utils import mymatch
object_info = mymatch(
    masked_object_info,
    lambda: empty_object_info(),
    lambda x: x
)
console.print(object_info)


ObjectInfo(
    parent_obj=Array([-1, -1], dtype=int32),
    parent_face=Array([0, 0], dtype=int32),
    child_face=Array([2, 4], dtype=int32),
    category_index=Array([6, 5], dtype=int32),
    pose=Array([[[ 0.21296251,  0.8599319 ,  0.4638577 , -3.6455798 ],
        [ 0.11613947,  0.44910288, -0.88589954, -2.5233161 ],
        [-0.97013324,  0.24253568, -0.00422996, -3.2954419 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.5552132 ,  0.14355308, -0.8192253 , -3.0910385 ],
        [ 0.40515423,  0.8135369 ,  0.4171414 , -3.0263913 ],
        [ 0.72635233, -0.56351525,  0.39352548,  1.8770814 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]],      dtype=float32),
    params=Array([[-0.36309052, -0.07025099,  1.5729983 ],
       [-0.8348894 ,  0.2492876 ,  1.419142  ]], dtype=float32)
)

In [ ]:
camera_pose = jnp.eye(4)

In [ ]:
valid_box_dims = jnp.where(
    (object_info.category_index == -1)[:, None],
    jnp.zeros(3),
    all_box_dims[object_info.category_index]
)

In [ ]:
poses = b.scene_graph.poses_from_scene_graph(
        object_info.pose,
        valid_box_dims,
        object_info.parent_obj, object_info.params,
        object_info.parent_face, object_info.child_face
    )

In [ ]:
jnp.linalg.inv(camera_pose) @ poses

Array([[[ 0.21296251,  0.8599319 ,  0.4638577 , -3.6455798 ],
        [ 0.11613947,  0.44910288, -0.88589954, -2.5233161 ],
        [-0.97013324,  0.24253568, -0.00422996, -3.2954419 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.5552132 ,  0.14355308, -0.8192253 , -3.0910385 ],
        [ 0.40515423,  0.8135369 ,  0.4171414 , -3.0263913 ],
        [ 0.72635233, -0.56351525,  0.39352548,  1.8770814 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]],      dtype=float32)

In [ ]:
object_info

ObjectInfo(parent_obj=Array([-1, -1], dtype=int32), parent_face=Array([0, 0], dtype=int32), child_face=Array([2, 4], dtype=int32), category_index=Array([6, 5], dtype=int32), pose=Array([[[ 0.21296251,  0.8599319 ,  0.4638577 , -3.6455798 ],
        [ 0.11613947,  0.44910288, -0.88589954, -2.5233161 ],
        [-0.97013324,  0.24253568, -0.00422996, -3.2954419 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],

       [[ 0.5552132 ,  0.14355308, -0.8192253 , -3.0910385 ],
        [ 0.40515423,  0.8135369 ,  0.4171414 , -3.0263913 ],
        [ 0.72635233, -0.56351525,  0.39352548,  1.8770814 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]],      dtype=float32), params=Array([[-0.36309052, -0.07025099,  1.5729983 ],
       [-0.8348894 ,  0.2492876 ,  1.419142  ]], dtype=float32))

In [ ]:
rendered = b.RENDERER.render(jnp.linalg.inv(camera_pose) @ poses, object_info.category_index)[..., :3]
